In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_parquet("../Dataset/NF-UQ-NIDS/NF-UQ-NIDS-V2_train.parquet")

In [3]:
df_valid = pd.read_parquet("../Dataset/NF-UQ-NIDS/NF-UQ-NIDS-V2_valid.parquet")

In [4]:
df_test = pd.read_parquet("../Dataset/NF-UQ-NIDS/NF-UQ-NIDS-V2_test.parquet")

In [5]:
load_X_df_train = df_train.drop(["Label"],axis=1)
Y_df_train = df_train["Label"]

In [6]:
load_X_df_valid = df_valid.drop(["Label"],axis=1)
Y_df_valid = df_valid["Label"]

In [7]:
load_X_df_test = df_test.drop(["Label"],axis=1)
Y_df_test = df_test["Label"]

In [8]:
from sklearn.preprocessing import QuantileTransformer

In [9]:
scaler_df = QuantileTransformer(output_distribution='normal')

In [10]:
X_df_train = scaler_df.fit_transform(load_X_df_train)

In [11]:
X_df_valid = scaler_df.fit_transform(load_X_df_valid)

In [12]:
X_df_test = scaler_df.fit_transform(load_X_df_test)

In [13]:
from keras.models import Sequential
from keras.layers import Dense

2025-11-17 17:50:50.657310: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-17 17:50:50.660701: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-17 17:50:50.669040: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-17 17:50:50.683406: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-17 17:50:50.687901: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-17 17:50:50.700406: I tensorflow/core/platform/cpu_feature_gu

In [14]:
from keras.metrics import Recall, Precision

In [15]:
def fit_model(trainX, trainy):
    model = Sequential(name="N5_model")
    model.add(Dense(9, input_dim=39, activation='relu', name="N5_i"))
    model.add(Dense(7, activation='relu', name="N5_l1"))
    model.add(Dense(5, activation='relu', name="N5_l2"))
    model.add(Dense(3, activation='relu', name="N5_l3"))
    model.add(Dense(1, activation='sigmoid', name="N5_o")) 
    model._name="N5_m"
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', Recall(), Precision()])
    
    model.fit(trainX, trainy, epochs=10, batch_size=1000, verbose=1, validation_data=(X_df_valid, Y_df_valid))
    return model

In [16]:
model = fit_model(X_df_train, Y_df_train)

filename = './model_N5.keras'
model.save(filename)
print('>Saved %s' % filename)

/home/awardana/miniconda3/envs/cids_env/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


2025-11-17 17:50:54.866172: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 6159405408 exceeds 10% of free system memory.


39484/39484 ━━━━━━━━━━━━━━━━━━━━ 50s 1ms/step - accuracy: 0.9597 - loss: 0.1247 - precision: 0.9657 - recall: 0.9751 - val_accuracy: 0.9747 - val_loss: 0.0764 - val_precision: 0.9763 - val_recall: 0.9862
Epoch 2/10
39484/39484 ━━━━━━━━━━━━━━━━━━━━ 48s 1ms/step - accuracy: 0.9750 - loss: 0.0738 - precision: 0.9771 - recall: 0.9857 - val_accuracy: 0.9750 - val_loss: 0.0732 - val_precision: 0.9777 - val_recall: 0.9852
Epoch 3/10
39484/39484 ━━━━━━━━━━━━━━━━━━━━ 48s 1ms/step - accuracy: 0.9754 - loss: 0.0723 - precision: 0.9775 - recall: 0.9859 - val_accuracy: 0.9706 - val_loss: 0.0803 - val_precision: 0.9779 - val_recall: 0.9782
Epoch 4/10
39484/39484 ━━━━━━━━━━━━━━━━━━━━ 48s 1ms/step - accuracy: 0.9743 - loss: 0.0750 - precision: 0.9764 - recall: 0.9855 - val_accuracy: 0.9752 - val_loss: 0.0739 - val_precision: 0.9756 - val_recall: 0.9877
Epoch 5/10
39484/39484 ━━━━━━━━━━━━━━━━━━━━ 48s 1ms/step - accuracy: 0.9754 - loss: 0.0726 - precision: 0.9763 - recall: 0.9872 - val_accuracy: 0.9724 

In [17]:
from keras.models import load_model
from numpy import argmax
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [18]:
filename_model = './model_N5.keras'
loaded_model = load_model(filename_model)

In [19]:
score = loaded_model.evaluate(X_df_test, Y_df_test, verbose=1)

print('Test loss:', score[0])
print('Test accuracy:', score[1])
print('Test recall:', score[2])
print('Test precision:', score[3])

# Calculate F1-score
precision = score[3]
recall = score[2]
f1_score = 2 * (precision * recall) / (precision + recall)

print('Test F1-score:', f1_score)


2025-11-17 17:59:06.241674: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2639745108 exceeds 10% of free system memory.


528796/528796 ━━━━━━━━━━━━━━━━━━━━ 425s 803us/step - accuracy: 0.9594 - loss: 0.1449 - precision: 0.9747 - recall: 0.9643
Test loss: 0.14520351588726044
Test accuracy: 0.9593233466148376
Test recall: 0.9641773104667664
Test precision: 0.974759042263031
Test F1-score: 0.969439301500685
